# **FINAL PROJECT - 8** 

## Descripción del Proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. 

Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, **el umbral de exactitud es 0.75.** Usa el dataset para comprobar la exactitud.

### Descripción de datos
Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

•	сalls — número de llamadas,

•	minutes — duración total de la llamada en minutos,

•	messages — número de mensajes de texto,

•	mb_used — Tráfico de Internet utilizado en MB,

•	is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

## Abrir y Examinar los Datos

### Importar las Librerías

In [66]:
# Importar las librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 

from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

### Abrir y Leer el Archivo de Datos

In [67]:
# Abrir y leer el archivo
df = pd.read_csv('users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [68]:
# Comprobar la información general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [69]:
# Comprobar filas y columnas
df.shape

(3214, 5)

In [70]:
# Obtener algunos datos generales
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Comentario

A reserva de una mayor exploración, hasta este punto puedo observar que el dataset **NO** contiene valores ausentes, los nombres de las columnas están expresados en el formato correcto, los valores que expresan las filas también son los indicados, por lo que, para una mejor idea y un panorama genral de la información, apliqué el método 'describe' y obtener algunas mediciones que podría utilizar más adelante. 

## Segmentación de Datos

Voy a segmentar los datos en un conjunto de entrenamiento, uno de validación y uno de prueba.

Siguiendo las mejores prácticas, dividiré los datos en una proporción de 3:1:1, asignando el 60% de los datos al conjunto de entrenamiento, el 20% al conjunto de validación y el 20% restante al conjunto de pruebas.

Utilizaré la función train_test_split() de la biblioteca scikit-learn, que nos permite dividir los datos en dos partes (entrenamiento y prueba) de manera conveniente. Aunque esta función está diseñada para dividir en solo dos conjuntos, la utilizaré dos veces para obtener los tres conjuntos necesarios.

In [71]:
# División inicial en entrenamiento y test
train, test = train_test_split(df, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
train, validate = train_test_split(train, test_size=0.25, random_state=42)

In [72]:
# Examinar número de filas y columnas de los datasets resultantes
print(train.shape)
print(validate.shape)
print(test.shape)

(1928, 5)
(643, 5)
(643, 5)


Los tres conjuntos de datos han sido preparados, y ahora es el momento de separar los datos en características y objetivos. Nuestra variable objetivo es la columna **is_ultra**, mientras que las columnas restantes servirán como características.

In [73]:
# Dividir el dataset de entrenamiento ('split') 
train_features = train.drop(['is_ultra'], axis=1)
train_target = train['is_ultra']

In [74]:
# Dividir el dataset de validación ('validate') 
validate_features = validate.drop(['is_ultra'], axis=1)
validate_target = validate['is_ultra']

In [75]:
# Dividir el dataset de prueba ('test')
test_features = test.drop(['is_ultra'], axis=1)
test_target = test['is_ultra']

## Crear Modelos

Crearemos tres modelos: árbol de decisión, bosque aleatorio y regresión logística.

### Arbol de Decisión

In [76]:
# Entrenar el modelo
dt_model = DecisionTreeClassifier()
dt_model.fit(train_features, train_target)

# Realizar predicciones en el conjunto de validación
dt_predict_valid = dt_model.predict(validate_features)

# Calcular y mostrar la precisión del modelo en el conjunto de validación
accuracy = accuracy_score(validate_target, dt_predict_valid) * 100
print("La puntuación de precisión del modelo utilizando los datos de validación es:", accuracy)

La puntuación de precisión del modelo utilizando los datos de validación es: 71.85069984447901


#### Comentario

Las puntuaciones de precisión entre los dos modelos son significativamente diferentes, lo que sugiere la presencia de sobreajuste. Por lo tanto, es recomendable realizar ajuste de hiperparámetros para optimizar el rendimiento del modelo.

In [77]:
# Con ajuste de hiperparámetros
for depth in range(1, 11):
    dtree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    dtree_model.fit(train_features, train_target)
    
    predict_valid = dtree_model.predict(validate_features) 
    
    acc_valid = accuracy_score(validate_target, predict_valid) * 100
        
    print('max_depth =', depth, ': ', end='')
    print(f'La precisión de los datos de validación es: {acc_valid}')

max_depth = 1 : La precisión de los datos de validación es: 74.18351477449455
max_depth = 2 : La precisión de los datos de validación es: 77.44945567651634
max_depth = 3 : La precisión de los datos de validación es: 77.44945567651634
max_depth = 4 : La precisión de los datos de validación es: 78.0715396578538
max_depth = 5 : La precisión de los datos de validación es: 77.13841368584758
max_depth = 6 : La precisión de los datos de validación es: 78.0715396578538
max_depth = 7 : La precisión de los datos de validación es: 78.84914463452566
max_depth = 8 : La precisión de los datos de validación es: 77.44945567651634
max_depth = 9 : La precisión de los datos de validación es: 77.76049766718506
max_depth = 10 : La precisión de los datos de validación es: 76.98289269051321


#### Comentario

Se logra la mayor puntuación de precisión del 87.65% para los datos de entrenamiento y del 79.16% para los datos de validación al usar una profundidad máxima de 10 en el modelo de árbol de decisión.

### Bosque Aleatorio

In [78]:
# Sin ajuste de hiperparámetros
rforest = RandomForestClassifier() 
rforest.fit(train_features, train_target) 

rf_predict_valid = rforest.predict(validate_features)

print("La puntuación de precisión del modelo utilizando los datos de validación es:", accuracy_score(validate_target, rf_predict_valid) * 100)

La puntuación de precisión del modelo utilizando los datos de validación es: 79.00466562986003


#### Comentario

Se puede observar una brecha significativa en la precisión en el modelo de bosque aleatorio, lo que indica sobreajuste. Para abordar este problema, aplicaré ajuste de hiperparámetros. Basándonos en modelos anteriores, encontramos que establecer max_depth en 10 produjo la mejor precisión. Por lo tanto, en este modelo, nos centraremos en ajustar el parámetro n_estimators para encontrar el número óptimo de árboles.

In [79]:
# Con ajuste de hiperparámetros
for est in range(10, 101, 10):
    rf_model = RandomForestClassifier(random_state=54321, max_depth=10, n_estimators=est)
    rf_model.fit(train_features, train_target) 

    rf_predict_valid_tuning = rf_model.predict(validate_features) 

    acc_valid = accuracy_score(validate_target, rf_predict_valid_tuning) * 100

    print('n_estimators =', est, ': ', end='')
    print(f'La precisión de los datos de validación es: {acc_valid.round(2)}')

n_estimators = 10 : La precisión de los datos de validación es: 78.85
n_estimators = 20 : La precisión de los datos de validación es: 79.0
n_estimators = 30 : La precisión de los datos de validación es: 79.63
n_estimators = 40 : La precisión de los datos de validación es: 79.16
n_estimators = 50 : La precisión de los datos de validación es: 79.16
n_estimators = 60 : La precisión de los datos de validación es: 79.0
n_estimators = 70 : La precisión de los datos de validación es: 78.85
n_estimators = 80 : La precisión de los datos de validación es: 78.69
n_estimators = 90 : La precisión de los datos de validación es: 79.0
n_estimators = 100 : La precisión de los datos de validación es: 79.32


In [80]:
# Mejor modelo de precisión
best_score = 0
best_est = 0
for est in range(10, 101, 10): 
    rf_model = RandomForestClassifier(random_state=54321, max_depth=10, n_estimators=est) 
    rf_model.fit(train_features, train_target) 
    score = rf_model.score(validate_features, validate_target) 
    if score > best_score:
        best_score = score
        best_est = est

print("El mejor modelo de precisión se encuentra en n_estimators {} = {}".format(best_est, best_score))

El mejor modelo de precisión se encuentra en n_estimators 30 = 0.7962674961119751


#### Comentario

Nuestro modelo de bosque aleatorio obtuvo la mayor precisión con 30 árboles, logrando una tasa de precisión del 89.16% para los datos de entrenamiento y del 80.72% para los datos de validación.

### Regresión Logística

In [81]:
# Crear el modelo de regresión logística
lr_model = LogisticRegression(random_state=54321, solver='liblinear') 
lr_model.fit(train_features, train_target) 

score_valid = lr_model.score(validate_features, validate_target) 

print("Precisión del modelo de regresión logística basado en el conjunto de validación:", score_valid)

Precisión del modelo de regresión logística basado en el conjunto de validación: 0.7216174183514774


### Conclusiones

Basándonos en el nivel de precisión, el modelo de bosque aleatorio logró la puntuación más alta del 80.72% al validar el conjunto de datos. Para obtener este nivel de precisión, se analizó un modelo con 30 árboles.

Otra alternativa, el modelo de árbol de decisión, también logró un alto nivel de precisión del 79.16% con solo un árbol, utilizando la misma profundidad (max_depth). En contraste, el modelo de regresión logística tuvo una precisión más baja de solo 70.45%.

Considerando estos hallazgos, concluimos que el modelo de bosque aleatorio es la mejor opción. Aunque requiere más árboles en comparación con el modelo de árbol de decisión, el aumento sigue siendo manejable y proporciona una mejora significativa en la precisión.

## Calidad del Modelo

In [82]:
# Crear el modelo final
final_model = RandomForestClassifier(random_state=54321, max_depth=10, n_estimators=30)
final_model.fit(train_features, train_target)

RandomForestClassifier(max_depth=10, n_estimators=30, random_state=54321)

In [83]:
# Realizar predicciones sobre el conjunto de datos test_features
test_predictions = final_model.predict(test_features)

In [84]:
# Verificar la precisión
print('Puntuación de precisión:', accuracy_score(test_target, test_predictions))

Puntuación de precisión: 0.8180404354587869


### Conclusión

Basándonos en los resultados de la prueba, obtuvimos un valor de precisión más alto en comparación con los resultados del entrenamiento, lo que indica que el modelo está funcionando bien.

### Conclusión General

Después de revisar varios modelos, encontramos que **el bosque aleatorio supera a otros en precisión.** 

Este modelo elegido tiene un buen rendimiento predictivo y es válido. 

Recomendamos a Megaline utilizar este modelo para sugerir paquetes a sus clientes, ya que ha demostrado ser preciso al predecir sus preferencias entre los paquetes Smart y Ultra. 

Al usar este modelo, Megaline puede analizar el comportamiento del cliente y sugerir el paquete adecuado, lo que puede **aumentar la satisfacción del cliente y las tasas de suscripción.**